In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

In [27]:
df = pd.read_excel('/Users/jinsuc28/Documents/GitHub/YearDream2/데이터분석프로젝트_VER2_210329_210926.xlsx')

In [28]:
df.head()

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970634,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,110500.0,NaN,VBANK,개발자 커리어,NaT
1,970650,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
2,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
3,970656,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
4,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


In [29]:
# 구매완료한 강의만 
df = df[df.state=='COMPLETED']
# 데이터 사이언스 == 데이터사이언스 로 바꾸기
df.loc[df.category_title=='데이터 사이언스', 'category_title']= '데이터사이언스'
# payment만 남기고 나머지 전처리
df = df[df.type=='PAYMENT']
# 0원인 강의만 전처리
df = df[df.transaction_amount!=0]
# 10000원 이하인 강의만 전처리
# df = df[df.transaction_amount>10000]
# category_title nan 값 제거 3개
df = df.loc[~(df['category_title'].isnull())]
# course_title, format 'B2B' 전처리
df = df.loc[~(df['course_title'].str.contains('B2B')|(df['format'].str.contains('B2B')))]
# 'B2B' 전처리 확인
df.loc[(df['course_title'].str.contains('B2B')|(df['format'].str.contains('B2B')))]
# 개발자 테스트용 3건 전처리
df = df[~(df['course_title'].str.contains('TEST|test'))]
# 기간 한정판매 강의 100건 전처리
df = df[~df['course_title'].str.contains('한정판매')]
df = df.loc[~(df['subcategory_title'].isnull())]
# format 올인원으로 먼저 파보기
df = df.loc[(df['course_title'].str.contains('올인원'))]

# 추천시스템 구현
* <font size=4>개요</font>
    * <font size=4> 유저의 구매 데이터를 바탕으로 코사인 유사도 기반 추천시스템 구현</font>
* <font size=4>방법론</font>
    * <font size=4>추천시스템의 대표적인 방법론 중 아이템기반 협업필터링 아이디어를 착안하였습니다.</font>
    * <font size=4>구매 이력을 강의 벡터의 점수로 판단하고 벡터를 만들었습니다.</font>

# 코사인 유사도

* <font size=4>정의</font>
    * <font size=4>내적공간의 두 벡터간 각도의 코사인값을 이용하여 측정된 벡터간의 유사한 정도</font>
    <br/>
    <br/>
    <br/>
    <br/>

$$ \cos\theta = \dfrac{x \cdot y}{|x| |y|}$$</font>

<br/>
<br/>
<font>$$
\text{cosine_sim}(i, j) = \frac{
\sum\limits_{u \in U_{ij}} r(u,i) \cdot r(u,j)}
{\sqrt{\sum\limits_{u \in U_{ij}} r(u,i)^2} \cdot
\sqrt{\sum\limits_{u \in U_{ij}} r(u,j)^2}
} 
$$</font>
<br/>
<br/>
<br/>

* <font size=4>용어 설명</font>
    * <font size=4>u(user): 패스트캠퍼스 강좌 구매자</font>
    * <font size=4>i,j: 패스트캠퍼스 강의</font>
    * <font size=4>i,j 사이 유사도를 비교하여 가장 높은 유사도 item(강의)을 추천하게됩니다.</font>



# 강의 추천시스템

In [30]:
# float을 int로 type 변경
df['customer_id'] = df['customer_id'].astype(int)

item = df.course_title.unique()
user = df.customer_id.unique()
user_rating = pd.DataFrame(index=user, columns=item)
user_rating.fillna(0, inplace = True)
item_rating = pd.DataFrame(index=item, columns=user)
item_rating.fillna(0, inplace = True)

* <font size=4>유저와 아이템을 가지고 item 기반 데이터프레임 만들기</font>

In [31]:
for row in df.itertuples():
    item_rating.loc[row.course_title, row.customer_id] +=1

* <font size=4>row: 강의, column: 유저로 하는 백터 만듭니다.</font>
* <font size=4>원본 데이터프레임을 행단위로 출력하며 새로운 데이터프레임에 유저가 강의를 들었으면 +1 해줍니다.</font>

In [32]:
item_rating

,478028,471759,478051,140846,474451,478101,80434,474806,453024,478189,...,670709,515805,670735,670747,670748,503113,670463,670787,578540,670833
올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 마디아의 UX/UI 포트폴리오,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 100억을 움직이는 단 10장의 PPT,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 김기현의 딥러닝을 활용한 자연어처리 입문,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 디지털 마케팅 MAX 2020,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
올인원 패키지 : 리마의 로맨스판타지 웹소설 일러스트,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 노잼봇과 함께 배우는 파이썬 유치원,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
올인원 패키지 : 스프링의 정석 : 남궁성과 끝까지 간다,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
올인원 패키지 : 클라우드를 활용한 데이터 파이프라인 구축,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* <font size=4>총 116개 강의, 16377명 유저가 존재</font>
* <font size=4>즉, 116개 벡터와 각 벡터의 16377개 성분이 존재</font>

# 코사인유사도 계산

In [ ]:
item_based_collabor = cosine_similarity(item_rating)
item_based_collabor

* <font size=4>116개 강의에 각각의 코사인 유사도를 계산하기</font>

# Shape 확인

In [9]:
print(item_based_collabor.shape)

(116, 116)


* <font size=4>116개 각 벡터간의 코사인 유사도 측정을 통해 116,116 행렬을 가집니다.</font>

In [10]:
item_based_collabor = pd.DataFrame(data=item_based_collabor, index =item_rating.index, columns = item_rating.index )
item_based_collabor

,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,올인원 패키지 : 마디아의 UX/UI 포트폴리오,올인원 패키지 : 100억을 움직이는 단 10장의 PPT,올인원 패키지 : 김기현의 딥러닝을 활용한 자연어처리 입문,올인원 패키지 : 디지털 마케팅 MAX 2020,올인원 패키지 : Android 앱 개발,올인원 패키지 : 인스타를 성장시킨 50명의 성공방정식,올인원 패키지 : C++ 실력완성,올인원 패키지 : 부동산 디벨로퍼,올인원 패키지 : 완전정복 시각디자인 기초,...,올인원 패키지 : SNS 콘텐츠 기획/제작 MAX 2020 + [20%할인쿠폰]초격차 패키지 : 무조건 팔리는 SNS 콘텐츠 끝장내기,올인원 패키지 : 디자인의 차이를 만드는 서체 활용법 by 산돌 서체 디자이너,올인원 패키지 : 공여사와 무작정 풀어보는 엑셀 실무,올인원 패키지 : 시선을 사로잡는 브랜드 디자인,올인원 패키지 : 최성일의 코딩 초급자를 위한 웹 모션 디자인,올인원 패키지 : 리마의 로맨스판타지 웹소설 일러스트,올인원 패키지 : 노잼봇과 함께 배우는 파이썬 유치원,올인원 패키지 : 스프링의 정석 : 남궁성과 끝까지 간다,올인원 패키지 : 클라우드를 활용한 데이터 파이프라인 구축,"올인원 패키지 : 디자인 기초 체력 다지기, 시각디자인 요소 활용"
올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,1.000000,0.000000,0.018344,0.0,0.009485,0.000000,0.014112,0.000000,0.007678,0.005799,...,0.0,0.000000,0.000000,0.009095,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
올인원 패키지 : 마디아의 UX/UI 포트폴리오,0.000000,1.000000,0.004169,0.0,0.000000,0.004319,0.008018,0.000000,0.000000,0.032949,...,0.0,0.012410,0.000000,0.020670,0.010419,0.006953,0.000000,0.000000,0.000000,0.013201
올인원 패키지 : 100억을 움직이는 단 10장의 PPT,0.018344,0.004169,1.000000,0.0,0.011310,0.013595,0.015144,0.008918,0.008239,0.008297,...,0.0,0.007813,0.013083,0.000000,0.006560,0.000000,0.000000,0.000000,0.009354,0.000000
올인원 패키지 : 김기현의 딥러닝을 활용한 자연어처리 입문,0.000000,0.000000,0.000000,1.0,0.000000,0.006170,0.000000,0.030354,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
올인원 패키지 : 디지털 마케팅 MAX 2020,0.009485,0.000000,0.011310,0.0,1.000000,0.004687,0.013051,0.003843,0.004733,0.008938,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.008497,0.000000,0.000000,0.007162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
올인원 패키지 : 리마의 로맨스판타지 웹소설 일러스트,0.000000,0.006953,0.000000,0.0,0.000000,0.000000,0.000000,0.007436,0.000000,0.003459,...,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
올인원 패키지 : 노잼봇과 함께 배우는 파이썬 유치원,0.000000,0.000000,0.000000,0.0,0.008497,0.010214,0.000000,0.000000,0.000000,0.007792,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
올인원 패키지 : 스프링의 정석 : 남궁성과 끝까지 간다,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.013375,0.007466,0.000000,0.011230,0.000000,0.000000,1.000000,0.016013,0.014228
올인원 패키지 : 클라우드를 활용한 데이터 파이프라인 구축,0.000000,0.000000,0.009354,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.015542,0.000000,0.023376,0.000000,0.000000,0.016013,1.000000,0.029617


* <font size=4>코사인 유사도 계산한 것을 데이터 프레임 만들기</font>
* <font size=4>대각성분에 경우 자기 자신에 대한 코사인 유사도이므로 1이 나옵니다.</font>

In [11]:
def item_collabor(course_title):
    return item_based_collabor.loc[course_title].sort_values(ascending=False)

* <font size=4>강의 명을 입력으로 넣은 후 입력 강의와 유사한 강의 순으로 출력하는 함수 만들기</font>


In [12]:
item_collabor('올인원 패키지 : Android 앱 개발').head(10)

올인원 패키지 : Android 앱 개발                                      1.000000
올인원 패키지 : iOS 앱 개발                                          0.056284
올인원 패키지 : 누적 다운로드 120만+ 1인 개발자와 함께하는 앱 개발 입문                0.031288
올인원 패키지 : AWS/Docker 클라우드 서버 구축                             0.025163
올인원 패키지 : C#과 유니티로 배우는 게임개발                                 0.020642
올인원 패키지 : 컴퓨터 공학 전공 필수                                      0.019201
올인원 패키지 : 네트워크와 보안 핵심 요약                                    0.016423
올인원 패키지 : 6개월 치 업무를 하루 만에 끝내는 업무자동화                         0.016061
올인원 패키지 : C#으로 하는 인공지능 게임 제작 : 강화학습 에이전트를 사용한 보드게임 제작 실습    0.015164
올인원 패키지 : 소비자 리서치와 보고서 작성                                   0.013938
Name: 올인원 패키지 : Android 앱 개발, dtype: float64

* <font size=4>유저가 특정 강의를 수강한다면 유사한 강의들을 추천해줄 수 있습니다.</font>

# 서브카테고리 추천시스템
* <font size=4>서브카테고리를 item으로 하는 추천시스템 만들기</font>

In [13]:
# customer_id float에서 int로 변경
df['customer_id'] = df['customer_id'].astype(int)
df = df[~(df.isnull())]
item = df.subcategory_title.unique()
user = df.customer_id.unique()
user_rating = pd.DataFrame(index=user, columns=item)
user_rating.fillna(0, inplace = True)
item_rating = pd.DataFrame(index=item, columns=user)
item_rating.fillna(0, inplace = True)

for i in df.itertuples():
    item_rating.loc[i.subcategory_title, i.customer_id] +=1

In [14]:
item_rating.head()

,478028,471759,478051,140846,474451,478101,80434,474806,453024,478189,...,670709,515805,670735,670747,670748,503113,670463,670787,578540,670833
PPT/보고서,1,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
UX/UI,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
딥러닝/인공지능,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
디지털마케팅,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
모바일앱,0,0,0,0,0,2,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [15]:
item_based_collabor = cosine_similarity(item_rating)
print(item_based_collabor.shape)

(32, 32)


In [16]:
item_based_collabor = pd.DataFrame(data=item_based_collabor, index =item_rating.index, columns = item_rating.index )

In [17]:
item_based_collabor

,PPT/보고서,UX/UI,딥러닝/인공지능,디지털마케팅,모바일앱,SNS마케팅,게임,부동산 개발,시각디자인,개발자 커리어,...,모션그래픽,업무자동화,리더십,일러스트,주식 투자,프론트엔드 개발,글쓰기/카피라이팅,데이터엔지니어링,백엔드 개발,VFX
PPT/보고서,1.000000,0.012986,0.012310,0.021796,0.009234,0.031947,0.012918,0.009182,0.015040,0.005089,...,0.002561,0.018661,0.033764,0.003922,0.020598,0.007258,0.013734,0.010227,0.000000,0.003983
UX/UI,0.012986,1.000000,0.006286,0.005674,0.015325,0.017673,0.005197,0.002032,0.040412,0.004504,...,0.014166,0.011471,0.025862,0.009763,0.004341,0.018739,0.005065,0.011315,0.014298,0.008813
딥러닝/인공지능,0.012310,0.006286,1.000000,0.009309,0.026074,0.009021,0.037899,0.001482,0.008667,0.027919,...,0.002066,0.006691,0.016763,0.003955,0.021366,0.017568,0.003693,0.055005,0.017377,0.006426
디지털마케팅,0.021796,0.005674,0.009309,1.000000,0.010087,0.066306,0.001283,0.008025,0.014082,0.002224,...,0.000000,0.015855,0.014186,0.003213,0.015000,0.000000,0.007502,0.000000,0.000000,0.000000
모바일앱,0.009234,0.015325,0.026074,0.010087,1.000000,0.020421,0.038112,0.000000,0.007395,0.026024,...,0.002518,0.014273,0.012770,0.004821,0.012539,0.022604,0.000000,0.013409,0.019063,0.011749
SNS마케팅,0.031947,0.017673,0.009021,0.066306,0.020421,1.000000,0.012786,0.000000,0.029241,0.001385,...,0.006970,0.022575,0.014139,0.008006,0.018688,0.006586,0.018692,0.009279,0.014657,0.000000
게임,0.012918,0.005197,0.037899,0.001283,0.038112,0.012786,1.000000,0.000000,0.008599,0.039718,...,0.002562,0.006224,0.005197,0.025506,0.007851,0.013315,0.002290,0.013644,0.010776,0.011955
부동산 개발,0.009182,0.002032,0.001482,0.008025,0.000000,0.000000,0.000000,1.000000,0.006724,0.000000,...,0.000000,0.000000,0.000000,0.001534,0.007674,0.000000,0.000000,0.000000,0.000000,0.000000
시각디자인,0.015040,0.040412,0.008667,0.014082,0.007395,0.029241,0.008599,0.006724,1.000000,0.002795,...,0.014063,0.017081,0.002377,0.016153,0.006284,0.008858,0.014666,0.012481,0.011829,0.018226
개발자 커리어,0.005089,0.004504,0.027919,0.002224,0.026024,0.001385,0.039718,0.000000,0.002795,1.000000,...,0.000000,0.007192,0.000000,0.010203,0.004253,0.031472,0.001985,0.002956,0.009339,0.003454


In [18]:
def item_collabor(sub_category):
    return item_based_collabor.loc[sub_category].sort_values(ascending=False)

In [19]:
item_collabor('머신러닝').head()

머신러닝        1.000000
딥러닝/인공지능    0.085051
데이터분석       0.076958
데이터엔지니어링    0.040288
개발자 커리어     0.028869
Name: 머신러닝, dtype: float64

* <font size=4>유저가 특정 서브카테고리 강의를 수강한다면 유사한 서브 카테고리들을 추천해줄 수 있습니다.</font>

# 개선방향
* <font size=4>비교적 희소행렬이 적은 데이터 수집을 통해 다양한 추천시스템 개선</font>
* <font size=4>메타 데이터들을 통해 컨텐츠 기반 추천시스템 구현</font>
* <font size=4>코사인 유사도는 두 벡터간의 각을 고려한 수치이기 때문에 scale이 무시되어 계산 따라서 다른 유사도 사용 개선</font>